<a href="https://colab.research.google.com/github/notAlex2/Translation-Team08-IFT6759/blob/master/notebooks/harman_bert_language_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

project_path = "/content/drive/My Drive/machine-translation"
os.chdir(project_path)

data_path = os.path.join(project_path, 'data')

In [0]:
! pip install transformers

In [0]:
import transformers
import numpy as np
import tensorflow as tf
import io
import tokenizers
import json
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import (
    PreTrainedTokenizer, 
    AutoTokenizer, 
    BertConfig, 
    TFBertModel
)
                          
from typing import Tuple

def set_seed(seed):
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seed(10)
unaligned_en_path = os.path.join(data_path, 'unaligned.en')

In [0]:
VOCAB_EXIST = True
VOCAB_SIZE = 30000
pretrained_tokenizer_path = "/content/drive/My Drive/machine-translation/transBERTo"
tokenizer_name = "bpe_tokenizer"

if not VOCAB_EXIST:
    # create vocab from scratch
    bpe_tokenizer = tokenizers.ByteLevelBPETokenizer()
    special_tokens = ["<s>","<pad>","</s>","<unk>","<mask>"]

    bpe_tokenizer.train(
                    files=[unaligned_en_path], 
                    vocab_size=VOCAB_SIZE, 
                    min_frequency=2, 
                    show_progress=True,
                    special_tokens=special_tokens
                    )

    # Save files to disk
    bpe_tokenizer.save(pretrained_tokenizer_path, tokenizer_name)
    
    # rename files to vocab.json and merges.txt
    src = os.path.join(pretrained_tokenizer_path, tokenizer_name + "-vocab.json")
    dst = os.path.join(pretrained_tokenizer_path, "vocab.json")
    os.rename(src, dst)

    src = os.path.join(pretrained_tokenizer_path, tokenizer_name + "-merges.txt")
    dst = os.path.join(pretrained_tokenizer_path, "merges.txt")
    os.rename(src, dst)

config = {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.3,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "num_attention_heads": 1,
  "num_hidden_layers": 1,
  "vocab_size": VOCAB_SIZE,
  "intermediate_size": 256,
  "max_position_embeddings": 256, 
  "model_type": "roberta" # roBERTa model is better than BERT for language modelling
}

config_path = os.path.join(pretrained_tokenizer_path, "config.json")
with open(config_path, 'w') as fp:
    json.dump(config, fp)

tokenizer = AutoTokenizer.from_pretrained(pretrained_tokenizer_path, cache_dir=None)

In [0]:
def mask_tokens(inputs: tf.Tensor, 
                tokenizer: PreTrainedTokenizer, 
                mlm_probability=0.15) -> Tuple[tf.Tensor, tf.Tensor]:
    """ Prepare masked tokens inputs/labels for masked language modeling: 80% MASK, 10% random, 10% original. """

    if tokenizer.mask_token is None:
        raise ValueError(
            "This tokenizer does not have a mask token which is necessary for masked language modeling"
        )
    
    labels = inputs # ideally this should be inputs.clone()

    # We sample a few tokens in each sequence for masked-LM training 
    # (with probability mlm_probability defaults to 0.15 in Bert/RoBERTa)
    probability_matrix = np.full(labels.shape, mlm_probability)
    # special_tokens_mask = [
    #         tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.numpy().tolist()
    #     ]
    special_tokens_mask = tokenizer.get_special_tokens_mask(inputs, already_has_special_tokens=True)
    probability_matrix = np.ma.array(probability_matrix, mask=special_tokens_mask).filled(0)
    if tokenizer._pad_token is not None:
        padding_mask = (labels == tokenizer.pad_token_id)
        probability_matrix = np.ma.array(probability_matrix, mask=padding_mask).filled(0)

    attention_masks = probability_matrix
    attention_masks[attention_masks > 0] = 1
    masked_indices = tf.compat.v1.distributions.Bernoulli(probs=probability_matrix)
    masked_indices = masked_indices.sample(1)[0].numpy()
    masked_indices = np.array(masked_indices, dtype=np.bool)
    # labels = labels.numpy() 
    # labels[~masked_indices] = -100 # We only compute loss on masked tokens

    # 80% of the time, we replace masked input tokens with tokenizer.mask_token ([MASK])
    choose_mask_tokens = tf.compat.v1.distributions.Bernoulli(probs=np.full(labels.shape, 0.8))
    choose_mask_tokens = choose_mask_tokens.sample(1)[0].numpy()
    choose_mask_tokens = np.array(choose_mask_tokens, dtype=np.bool)
    indices_replaced = choose_mask_tokens & masked_indices
    inputs = inputs.numpy()
    inputs[indices_replaced] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)

    # 10% of the time, we replace masked input tokens with random word
    choose_random_tokens = tf.compat.v1.distributions.Bernoulli(probs=np.full(labels.shape, 0.5))
    choose_random_tokens = choose_random_tokens.sample(1)[0].numpy()
    choose_random_tokens = np.array(choose_random_tokens, dtype=np.bool)
    indices_random = choose_random_tokens & masked_indices & ~indices_replaced
    indices_random, choose_random_tokens
    random_words = np.random.randint(0, len(tokenizer), labels.shape, dtype=np.int32)
    inputs[indices_random] = random_words[indices_random]

    # The rest of the time (10% of the time) we keep the masked input tokens unchanged
    return inputs, attention_masks, labels

In [0]:
# datalaoder stuff
def max_length(tensor):
    return max(len(t) for t in tensor)

def tokenize_string(tokenizer, raw_string):
    return tokenizer.encode(raw_string)
  
def encode_dataset(file_path, pretrained_tokenizer_path, num_examples):
    tokenizer = AutoTokenizer.from_pretrained(pretrained_tokenizer_path)
    sentences = io.open(file_path, encoding='UTF-8').read().strip().split('\n')
    encoded_sequences = [tokenize_string(tokenizer, sentence) for sentence in sentences[:num_examples]]
    encoded_sequences = pad_sequences(encoded_sequences, padding='post', value=tokenizer.pad_token_id)
    return encoded_sequences

NUM_EXAMPLES = 6 
BATCH_SIZE = 2
encoded_sequences = encode_dataset(unaligned_en_path, pretrained_tokenizer_path, NUM_EXAMPLES)

def data_generator_fn():
    for seq in encoded_sequences:
        inputs = tf.convert_to_tensor(seq)
        inputs, attention_masks, masked_labels =  mask_tokens(inputs, tokenizer, mlm_probability=0.15)
        yield (inputs, attention_masks), masked_labels

# dataset object
dataset = tf.data.Dataset.from_generator(
    data_generator_fn,
    output_types=((tf.int32, tf.int32), tf.int32)
    ).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [245]:
for inputs in dataset.take(1):
    print(inputs)

((<tf.Tensor: shape=(2, 71), dtype=int32, numpy=
array([[  0,   4, 527, 320,   4,   4, 156, 264,   4,   4,   4, 490,   4,
          4,   4,   4,   4,   4, 239,   4,  54,   4,   4,   4,   4,   4,
         16,   4,   4,  54,   4,   4,   4,   4, 401,   4, 342,   4, 504,
        652,   4,   4,   4,   4,   4,   2,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1],
       [  0,   4,   4,   4,   4, 655,   4,   4,   4,   4,   4,   4,   4,
          4,   4, 441,   4,   4,   4,   4,   4,   4,   4,   4,   4,   4,
          4,   4, 340,   4,   4,   4,  51,   4,   4,   4,   4, 366,   4,
          4,   4,   4,   4,   4, 268,   4,   4,   4,   4,   4,   4,   4,
          4,   4,   4, 422, 534,   4,   4,   4,   4, 355,   4, 587,   4,
          4,   4,  48,   4,   4,   2]], dtype=int32)>, <tf.Tensor: shape=(2, 71), dtype=int32, numpy=
array([[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
      

#### Train Model here

In [246]:
configuration = BertConfig.from_pretrained(pretrained_tokenizer_path)
bert_model = TFBertModel(config=configuration)

input = np.random.randint(0,10, size=(2,3))
output = bert_model(input)
output[0].shape, output[1].shape

(TensorShape([2, 3, 128]), TensorShape([2, 128]))

In [247]:
bert_model.summary()

Model: "tf_bert_model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  4022272   
Total params: 4,022,272
Trainable params: 4,022,272
Non-trainable params: 0
_________________________________________________________________


In [248]:
from transformers import TFBertForMaskedLM

configuration = BertConfig.from_pretrained("google/bert_uncased_L-10_H-128_A-2")
model = TFBertForMaskedLM(config=configuration)

input = np.random.randint(0,10, size=(2,3))
output = model(input)

output[0].shape, model.summary()

Model: "tf_bert_for_masked_lm_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  5972096   
_________________________________________________________________
mlm___cls (TFBertMLMHead)    multiple                  4020154   
Total params: 6,019,386
Trainable params: 6,019,386
Non-trainable params: 0
_________________________________________________________________


(TensorShape([2, 3, 30522]), None)

In [0]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
# loss = tf.keras.losses.CategoricalCrossentropy()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss)

In [256]:
model.fit((inputs, attention_masks), labels, epochs=10)

Epoch 1/10


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1/1 [==============================] - 0s 2ms/step - loss: 10.3212
Epoch 2/10
1/1 [==============================] - 0s 1ms/step - loss: 9.8468
Epoch 3/10
1/1 [==============================] - 0s 3ms/step - loss: 9.5937
Epoch 4/10
1/1 [==============================] - 0s 1ms/step - loss: 9.3391
Epoch 5/10
1/1 [==============================] - 0s 2ms/step - loss: 9.1358
Epoch 6/10
1/1 [==============================] - 0s 1ms/step - loss: 8.8983
Epoch 7/10
1/1 [==============================] - 0s 804us/step - loss: 8.6756
Epoch 8/10
1/1 [==============================] - 0s 2ms/step - loss: 8.4460
Epoch 9/10
1/1 [==============================] - 0s 3ms/step - loss: 8.2419
Epoch 10/10
1/1 [==============================] - 0s 874us/step - loss: 8.0255


In [0]:
@tf.function
def train_step(model, inputs, labels):
    with tf.GradientTape() as tape:
        outputs = model(inputs)
        loss = outputs[0]
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

In [0]:
for step, ((inputs, atttention_masks), labels) in enumerate(dataset):
    loss = train_step(model, inputs, labels)